In [66]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [67]:
# first set the working directory. This code will be changed based on the relative location of the data files 
# on the local drive of the computer executing the command. 
os.chdir('C:\\Users\\belincoln\\Documents\\! CBP\\!User Fees\\!! Goal 1 Dashboards')

# Works well for Jupyter Notebooks, can be configured in Spyder using file explorer. 
collections = pd.read_excel(os.path.join('Source Emails & Source Files','Files','Collections',
                                         'COBRA_Air','Collections cc495 - FY13 - FY18.xls'))

In [68]:
# Begin Data Cleaning
# delete columns and rows that contain only na
collections = collections.dropna(axis=0, how = 'all')
collections = collections.dropna(axis=1, how = 'all')

In [69]:
# Delete the first two rows and only keep the Period Column and Collection Columns
collections = collections.iloc[2:,[1,-1,-3,-4]]

In [70]:
# Make first row column headers
collections.columns = collections.iloc[0,:]

In [71]:
# Make sure all columns have a name! Rename first column 'Period'
# Delete first two rows
collections.columns.values[0] = 'Period'

collections = collections.iloc[2:,:]

In [72]:
# Delete rows that contain sums for each company (don't want to double count collections)
# Also delete additional NaNs
collections = collections.dropna(axis = 0, subset = ['Period'])

In [73]:
# Sum Interest, Penalty, and Principal Collection Amounts
collections['Collections'] = collections[['Applied Penalty Amount', 'Applied Interest Amount',
       'Applied Principal Amount']].sum(axis = 1)

In [74]:
# remove tilda from index (unclear why it exists in the first place)
collections['Period'] = collections['Period'].str.rstrip('~')
# remove extra space from index
collections['Period'] = collections['Period'].str.rstrip(' ')

In [75]:
# Groups Collections based on Collection Period, across all companies
collections = collections.groupby(collections['Period']).sum()

In [76]:
# Remove audit payments
collections= collections[~collections.index.str.contains("\*")]

# Add an additional column that shows remittance period (independent of year)
collections['Remittance Period'] = collections.index.str.split('20').str[0]


# Create Calendar Year Column
collections['Calendar Year'] = collections.index.str.split(')').str[1]
# Turn Years into integers
collections['Calendar Year'] = collections['Calendar Year'].astype(int)

# Filter on years not a part of analysis
years = [2012,2013,2014,2015,2016,2017,2018]
collections = collections[collections['Calendar Year'].isin(years)]

In [77]:
#remove collection data for which we don't have workload data
searchfor = ['Qtr 01 \(Jan-Mar\) 2012','Qtr 02 \(Apr-Jun\) 2012', 'Qtr 03 \(Jul-Sept\) 2012','Qtr 04 \(Oct-Dec\) 2018']
collections = collections[~collections.index.str.contains('|'.join(searchfor))]


In [78]:
#%% Read Workload Data
workload = pd.read_excel(os.path.join('Source Emails & Source Files','Files','Workload',
                                      'COBRA_Air','FY09-fy18-passenger data air and cruise.xlsx'))

In [79]:
# Drop Cruise and Total Rows
workload.drop([7,8],axis = 0,inplace=True)

In [80]:
workload['Data Id'] = workload['Data Id'] + ': ' + workload['Data Elements  - National']

In [81]:
workload.drop('Data Elements  - National', axis = 1, inplace = True)

In [82]:
workload['Data Id'] = workload['Data Id'].shift()
workload = workload.iloc[[1,3,5]]

In [83]:
workload

,Data Id,10/1/2008,11/1/2008,12/1/2008,1/1/2009,2/1/2009,3/1/2009,4/1/2009,5/1/2009,6/1/2009,...,12/1/2017,1/1/2018,2/1/2018,3/1/2018,4/1/2018,5/1/2018,6/1/2018,7/1/2018,8/1/2018,9/1/2018
1,USC00011: Commercial Aircraft Pax,5437977.0,4986218.0,5446923.0,5628757.0,4590526.0,5649832.0,5848890.0,5324342.0,5873692.0,...,7794441.0,8235566.0,6785954.0,8600528.0,8662139.0,8608627.0,9165631.0,10615039.0,10164128.0,8102772.0
3,MPP00013: Private Aircraft Passengers,31531.0,32307.0,29799.0,36279.0,27772.0,33705.0,29347.0,20400.0,24314.0,...,22546.0,25712.0,23464.0,30079.0,26404.0,22555.0,24388.0,26656.0,23076.0,20025.0
5,USCP0011: Preclear Air Ports Pax/Crew combined...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1596994.0,1565813.0,1530305.0,1884844.0,1745714.0,1680540.0,1845968.0,2073376.0,2034479.0,1651825.0


In [84]:
workload = workload.append(workload.sum(numeric_only = True), ignore_index = True)
workload = workload.transpose()

In [85]:
workload.iloc[0,-1] = 'Sum of Workload Ids'

In [86]:
workload.columns = workload.iloc[0]
workload.drop('Data Id', inplace = True, axis = 0)

In [87]:
# Create Calendar Year and Month Columns
workload['Month'] = workload.index.str.split('/').str[0]
workload['Calendar Year'] = workload.index.str.split('/').str[2]


In [88]:
# Filter on years not a part of analysis
years = ['2012','2013','2014','2015','2016','2017','2018']
workload = workload[workload['Calendar Year'].isin(years)]

In [89]:
# Build out Remittance Period Columns
conditions = [(workload['Month'] == '1'), (workload['Month'] == '2'), (workload['Month'] == '3'), 
              (workload['Month'] == '4'), (workload['Month'] == '5'), (workload['Month'] == '6'),
              (workload['Month'] == '7'),(workload['Month'] == '8'),(workload['Month'] == '9'),
              (workload['Month'] == '10'),(workload['Month'] == '11'),(workload['Month'] == '12')] 
choices = ['Qtr 01 (Jan-Mar)','Qtr 01 (Jan-Mar)','Qtr 01 (Jan-Mar)',
           'Qtr 02 (Apr-Jun)','Qtr 02 (Apr-Jun)','Qtr 02 (Apr-Jun)',
           'Qtr 03 (Jul-Sept)','Qtr 03 (Jul-Sept)', 'Qtr 03 (Jul-Sept)',
           'Qtr 04 (Oct-Dec)','Qtr 04 (Oct-Dec)','Qtr 04 (Oct-Dec)']
workload['Remittance Period'] = np.select(conditions, choices, default='error')

In [90]:
# Reset index to Remittance Period plus Calendar Year to merge with Collection data
workload.index = workload['Remittance Period'] + ' ' + workload['Calendar Year']
# Drop unnecssary columns
workload.drop(['Calendar Year', 'Month','Remittance Period'], inplace = True, axis=1)


In [91]:
# Sum on Remittance Period
workload = workload.groupby(workload.index).sum()

In [92]:
workload_collections = pd.merge(workload,collections,how = 'inner', left_index = True, right_index = True)
#%%

In [93]:
len(workload_collections)

24

In [94]:
corr = workload_collections.corr()

In [95]:
corr

,USC00011: Commercial Aircraft Pax,MPP00013: Private Aircraft Passengers,USCP0011: Preclear Air Ports Pax/Crew combined (Calc),Sum of Workload Ids,Collections,Calendar Year
USC00011: Commercial Aircraft Pax,1.000000,-0.292130,0.947597,0.998886,-0.109670,0.713713
MPP00013: Private Aircraft Passengers,-0.292130,1.000000,-0.339901,-0.299855,0.370522,-0.384099
USCP0011: Preclear Air Ports Pax/Crew combined (Calc),0.947597,-0.339901,1.000000,0.961612,-0.052793,0.852463
Sum of Workload Ids,0.998886,-0.299855,0.961612,1.000000,-0.101463,0.738833
Collections,-0.109670,0.370522,-0.052793,-0.101463,1.000000,0.177910
Calendar Year,0.713713,-0.384099,0.852463,0.738833,0.177910,1.000000


In [96]:
workload_collections

,USC00011: Commercial Aircraft Pax,MPP00013: Private Aircraft Passengers,USCP0011: Preclear Air Ports Pax/Crew combined (Calc),Sum of Workload Ids,Collections,Remittance Period,Calendar Year
Qtr 01 (Jan-Mar) 2013,18700855.0,83857.0,4006973.0,22791685.0,1.446142e+08,Qtr 01 (Jan-Mar),2013
Qtr 01 (Jan-Mar) 2014,19219847.0,83617.0,4162697.0,23466161.0,1.519669e+08,Qtr 01 (Jan-Mar),2014
Qtr 01 (Jan-Mar) 2015,20244415.0,83705.0,4519727.0,24847847.0,1.626936e+08,Qtr 01 (Jan-Mar),2015
Qtr 01 (Jan-Mar) 2016,21847254.0,84635.0,4635919.0,26567808.0,1.683570e+08,Qtr 01 (Jan-Mar),2016
Qtr 01 (Jan-Mar) 2017,22369440.0,77939.0,4762410.0,27209789.0,1.820994e+08,Qtr 01 (Jan-Mar),2017
Qtr 01 (Jan-Mar) 2018,23622048.0,79255.0,4980962.0,28682265.0,1.993085e+08,Qtr 01 (Jan-Mar),2018
Qtr 02 (Apr-Jun) 2013,20632109.0,81847.0,3941004.0,24654960.0,1.369273e+08,Qtr 02 (Apr-Jun),2013
Qtr 02 (Apr-Jun) 2014,21778636.0,84433.0,4376274.0,26239343.0,1.428367e+08,Qtr 02 (Apr-Jun),2014
Qtr 02 (Apr-Jun) 2015,22657474.0,82242.0,4615960.0,27355676.0,1.545880e+08,Qtr 02 (Apr-Jun),2015
Qtr 02 (Apr-Jun) 2016,23744750.0,80985.0,4720735.0,28546470.0,1.629744e+08,Qtr 02 (Apr-Jun),2016


In [97]:
%store workload_collections

Stored 'workload_collections' (DataFrame)
